In [43]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import os 
import pickle
import json
from sklearn.metrics import mean_squared_error, r2_score




In [31]:
# recup les data raw

data_raw = pd.read_csv('../../data/raw_data/raw.csv')

display(data_raw)

,ave_flot_air_flow,ave_flot_level,iron_feed,starch_flow,amina_flow,ore_pulp_flow,ore_pulp_pH,ore_pulp_density,silica_concentrate
2017-04-24 00:00:00,300.263166,383.982443,55.17,1979.589150,599.676489,400.017222,9.774028,1.753206,4.360000
2017-04-24 01:00:00,299.782402,386.049069,55.17,1758.466329,600.043100,400.484528,9.539246,1.754190,3.290000
2017-04-24 02:00:00,299.750052,385.250935,55.17,2379.752428,599.948406,400.325617,9.434227,1.756873,4.900000
2017-04-24 03:00:00,299.997522,389.635519,55.17,2287.130046,599.580383,399.801506,9.725607,1.727125,4.860153
2017-04-24 04:00:00,300.005220,387.810807,55.17,2291.789167,599.871217,399.567333,9.845198,1.633063,4.780898
...,...,...,...,...,...,...,...,...,...
2017-08-27 10:00:00,299.958702,466.646630,60.02,3191.856167,319.003256,402.920396,9.294575,1.559673,1.690000
2017-08-27 11:00:00,300.013584,468.726893,60.02,3460.924500,393.149528,397.119469,9.393422,1.696894,1.590000
2017-08-27 12:00:00,300.127863,490.918520,60.02,3382.668556,430.495100,400.424371,9.367542,1.721010,1.450000
2017-08-29 22:00:00,299.670466,445.903411,54.49,2778.946000,563.556606,398.260061,8.844935,1.689140,2.970000


In [32]:
# tratement de cette donné 
# SPLIT

def Splite(Data_Raw = pd.read_csv('../../data/raw_data/raw.csv')):

    data_raw = Data_Raw

    # Séparation des variable cible et des Features
    df_y = data_raw[['silica_concentrate']]
    df_X = data_raw.drop('silica_concentrate', axis=1) 
    
    # Split en 4 DF distincts 

    X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)

    path = "../../data/processed_data"

    # Enregistrer les DataFrames en fichiers CSV
    X_train.to_csv(os.path.join(path, 'X_train.csv'), index=False)
    X_test.to_csv(os.path.join(path, 'X_test.csv'), index=False)
    y_train.to_csv(os.path.join(path, 'y_train.csv'), index=False)
    y_test.to_csv(os.path.join(path, 'y_test.csv'), index=False)


    return print ("La création des fichier train et test s'est bien éfféctuée avec une repartition de 80/20%.")

Splite()

La création des fichier train et test s'est bien éfféctuée avec une repartition de 80/20%.


In [34]:
def normalize_data(X_train_path='../../data/processed_data/X_train.csv', X_test_path='../../data/processed_data/X_test.csv'):
    # Chargement des données
    X_train = pd.read_csv(X_train_path)
    X_test = pd.read_csv(X_test_path)

    # Normalisation des données
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Conversion en DataFrame pour la sauvegarde
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
  
    # Définir le chemin où les fichiers CSV seront sauvegardés
    path = "../../data/processed_data"
 
    # Enregistrer les DataFrames normalisés en fichiers CSV
    X_train_scaled_df.to_csv(os.path.join(path, 'X_train_scaled.csv'), index=False)
    X_test_scaled_df.to_csv(os.path.join(path, 'X_test_scaled.csv'), index=False)

    return print("Normalisation des données effectuée et fichiers sauvegardés.")

normalize_data()

Normalisation des données effectuée et fichiers sauvegardés.


In [41]:
# 3 - Grid shearch 

def grid_search_decision_tree(X_train_path='../../data/processed_data/X_train_scaled.csv', y_train_path='../../data/processed_data/y_train.csv'):
    # Chargement des données
    X_train = pd.read_csv(X_train_path)
    y_train = pd.read_csv(y_train_path)

    # Définition de l'arbre de décision
    model = DecisionTreeRegressor()

    # Définition des paramètres à tester
    param_grid = {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        'max_depth': [None, 5, 10, 15, 20] }

    # Configuration de la recherche de grille
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                               scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=1)

    # Exécution de la recherche de grille
    grid_search.fit(X_train, y_train)

    # Sauvegarde du meilleur modèle
    models_path = "../../models"
    if not os.path.exists(models_path):
        os.makedirs(models_path)
        
    best_model_filename = os.path.join(models_path, 'best_decision_tree_model.pkl')
    with open(best_model_filename, 'wb') as file:
        pickle.dump(grid_search.best_estimator_, file)

    return print(f"Meilleur modèle sauvegardé dans {models_path}/best_decision_tree_model.pkl")

grid_search_decision_tree()



Fitting 5 folds for each of 20 candidates, totalling 100 fits
Meilleur modèle sauvegardé dans ../../models/best_decision_tree_model.pkl


In [42]:
# entrainement du modele
def train_model(X_train_path='../../data/processed_data/X_train_scaled.csv', 
                y_train_path='../../data/processed_data/y_train.csv',
                model_path='../../models/best_decision_tree_model.pkl'):
    
    # Chargement des données
    X_train = pd.read_csv(X_train_path)
    y_train = pd.read_csv(y_train_path)

    # Chargement du meilleur modèle depuis le fichier pickle
    with open(model_path, 'rb') as file:
        best_model = pickle.load(file)

    # Entraînement du modèle avec les meilleures hyperparamètres
    best_model.fit(X_train, y_train)

    # Sauvegarde du modèle entraîné
    trained_model_filename = os.path.join(os.path.dirname(model_path), 'trained_decision_tree_model.pkl')
    with open(trained_model_filename, 'wb') as file:
        pickle.dump(best_model, file)

    return print(f"Modèle entraîné sauvegardé dans {trained_model_filename}")

train_model()

Modèle entraîné sauvegardé dans ../../models\trained_decision_tree_model.pkl


In [44]:
def evaluation(X_test_path='../../data/processed_data/X_test_scaled.csv', 
               y_test_path='../../data/processed_data/y_test.csv',
               model_path='../../models/trained_decision_tree_model.pkl', 
               predictions_path='../../data/predictions.csv', 
               metrics_path='../../metrics/scores.json'):
    
    # Chargement des données
    X_test = pd.read_csv(X_test_path)
    y_test = pd.read_csv(y_test_path)

    # Chargement du modèle entraîné
    with open(model_path, 'rb') as file:
        trained_model = pickle.load(file)

    # Prédictions
    predictions = trained_model.predict(X_test)

    # Calcul des métriques d'évaluation
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    # Sauvegarde des prédictions dans un DataFrame
    predictions_df = pd.DataFrame({'Actual': y_test.squeeze(), 'Predicted': predictions})
    predictions_df.to_csv(predictions_path, index=False)

    # Sauvegarde des métriques dans un fichier JSON
    metrics = {'MSE': mse, 'R2': r2}
    with open(metrics_path, 'w') as json_file:
        json.dump(metrics, json_file)

    print(f"Prédictions sauvegardées dans {predictions_path}")
    return    print(f"Métriques d'évaluation sauvegardées dans {metrics_path}")


evaluation()

Prédictions sauvegardées dans ../../data/predictions.csv
Métriques d'évaluation sauvegardées dans ../../metrics/scores.json
